In [1]:
# 신경망 구성을 손쉽게 해주는 tensorflow.layers를 사용
# CNN.ipynb 파일을 재구성한 것이니 소스를 한 번 비교 하면 좋을듯
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [2]:
#######
# 신경망 모델 구성
#######
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])
is_training = tf.placeholder(tf.bool)

# 기본적으로 inputs, outputs size, kernel size 만 넣어주면
# 활성화 함수 적용은 물론, 컨볼루션 신경망을 만들기 위한 나머지 수치들은 알아서 계산
L1 = tf.layers.conv2d(X, 32, [3,3])
L1 = tf.layers.max_pooling2d(L1, [2,2], [2,2])
L1 = tf.layers.dropout(L1, 0.7, is_training)

L2 = tf.layers.conv2d(L1, 64, [3,3])
L2 = tf.layers.max_pooling2d(L2, [2,2], [2,2])
L2 = tf.layers.dropout(L2, 0.7, is_training)

L3 = tf.contrib.layers.flatten(L2)
L3 = tf.layers.dense(L3, 256, activation=tf.nn.relu)
L3 = tf.layers.dropout(L3, 0.5, is_training)

model = tf.layers.dense(L3, 10, activation=None)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

In [4]:
#######
# 신경망 모델 학습
#######
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

batch_size=100
total_batch = int(mnist.train.num_examples/batch_size)

for epoch in range(15):
    total_cost = 0
    
    for i in range(15):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape(-1, 28, 28, 1)
        _, cost_val = sess.run([optimizer, cost], feed_dict={X: batch_xs, Y: batch_ys, is_training:True})
        
        total_cost += cost_val
        
    print('Epoch : ', '%04d' %(epoch+1), 'Avg. cost = ', '{:.4f}'.format(total_cost / total_batch))
    
print ('최적화 완료')

Epoch :  0001 Avg. cost =  0.0394
Epoch :  0002 Avg. cost =  0.0148
Epoch :  0003 Avg. cost =  0.0101
Epoch :  0004 Avg. cost =  0.0085
Epoch :  0005 Avg. cost =  0.0067
Epoch :  0006 Avg. cost =  0.0066
Epoch :  0007 Avg. cost =  0.0064
Epoch :  0008 Avg. cost =  0.0055
Epoch :  0009 Avg. cost =  0.0044
Epoch :  0010 Avg. cost =  0.0045
Epoch :  0011 Avg. cost =  0.0039
Epoch :  0012 Avg. cost =  0.0036
Epoch :  0013 Avg. cost =  0.0033
Epoch :  0014 Avg. cost =  0.0039
Epoch :  0015 Avg. cost =  0.0030
최적화 완료


In [5]:
#######
# 결과 확인
#######
is_correct = tf.equal(tf.argmax(model,1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도 : ', sess.run(accuracy, feed_dict={X: mnist.test.images.reshape(-1, 28,28,1), Y: mnist.test.labels, is_training:False}))

정확도 :  0.9674
